In [ ]:
import torch
from data_logger import DataLogger
from dqn_agent import Agent 
from utils import *
from tqdm import tqdm
print(torch.cuda.is_available())

In [ ]:
#Params environment
game="mario"
level="SuperMarioBros-1-1-v0"
action_space="SIMPLE_MOVEMENT"

skip = 4 
gray_scale = True
frame_shape = (84, 84)
num_stack = 4

#Training model
model = "DDQN"
episodes = 20000
batch_size = 32
learning_rate = 0.00025

exploration_rate = 1 #initial value for epsilon
discount_factor = 0.99
epsilon_min = 0.1
epsilon_decay = 0.99999975

#Logging
n_time_steps_save_model=10000
n_episodes_average=3

In [ ]:
env = get_env(game=game, level=level, action_space=action_space)
env = apply_wrappers(env, skip=skip, gray_scale=gray_scale, shape=frame_shape, num_stack=num_stack)

agent = Agent(env, batch_size, learning_rate, observation_size=(num_stack,)+frame_shape, action_size=get_action_space_size(env), exploration_rate=exploration_rate, discount_factor=discount_factor)
logger = DataLogger(env, agent, model=model)

for episode in tqdm(range(episodes)):
    state = env.reset()

    while True:
        action_idx = agent.act(state)

        next_state, reward, done, info = env.step(get_action(action_idx, env))

        agent.append_experience_to_memory(state, next_state, action_idx, reward, done)
        
        loss = agent.learn()

        state = next_state

        env.render()

        if done or info["flag_get"]:
            break

        logger.time_step(reward, n_time_steps_save_model=n_time_steps_save_model)

    logger.episode_step(info, n_episodes_average=n_episodes_average)

logger.close()

In [ ]:
env.close()

In [ ]:
env = get_env(game=game, level=level, action_space=action_space)
env = apply_wrappers(env, skip=skip, gray_scale=gray_scale, shape=frame_shape, num_stack=num_stack)

agent = Agent(env, batch_size, learning_rate, observation_size=(num_stack,)+frame_shape, action_size=get_action_space_size(env), exploration_rate=exploration_rate, discount_factor=discount_factor)
agent.net.load_state_dict(torch.load('best_model.pth'))
time_stamp=0
state = env.reset()

while time_stamp<1e4:
        action_idx = agent.act(state)

        next_state, reward, done, info = env.step(get_action(action_idx, env))

        state = next_state

        env.render()
        
        time_stamp += 1

        if done or info["flag_get"]:
            break